In [1]:
import os
os.chdir("../")

In [38]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    params_num_encoder_layers: int
    params_num_tags: int
    params_vocab_size: int
    params_embedding_dim: int
    params_fully_connected_dim: int
    params_num_heads: int
    params_max_positional_encoding_length: int
    params_epochs: int
    params_batch_size: int



In [39]:
from named_entity_recognition.constants import *
from named_entity_recognition.utils.common import read_yaml, create_directories

In [40]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:

        config = self.config.model_trainer
        params = self.params
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = Path(config.root_dir),
            data_path = Path(config.data_path),
            params_num_encoder_layers=params.NUM_ENCODER_LAYERS,
            params_num_tags = params.NUM_TAGS,
            params_vocab_size = params.VOCAB_SIZE,
            params_embedding_dim = params.EMBEDDING_DIM,
            params_fully_connected_dim = params.FULLY_CONNECTED_DIM,
            params_num_heads = params.NUM_HEADS,
            params_max_positional_encoding_length = params.MAX_POSITIONAL_ENCODING_LENGTH,
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE
        )

        return model_trainer_config

In [41]:
import os
import pickle
from datasets import load_from_disk
from named_entity_recognition import logger
from named_entity_recognition.transformer.model import NERModel
from named_entity_recognition.transformer.optimizer import get_optimizer_with_custom_lr_sheduler
from named_entity_recognition.transformer.losses import MaskedLoss
from named_entity_recognition.transformer.metrics import masked_acc
import numpy as np
import tensorflow as tf

In [44]:
class ModelTrainer:

    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def get_model(self):

        model = NERModel(
            num_tags=self.config.params_num_tags,
            num_layers=self.config.params_num_encoder_layers,
            embedding_dim=self.config.params_embedding_dim,
            fully_connected_dim=self.config.params_fully_connected_dim,
            num_heads=self.config.params_num_heads,
            vocab_size=self.config.params_vocab_size,
            max_positional_encoding=self.config.params_max_positional_encoding_length
        )

        self.model = model


    def train(self):

        data = load_from_disk(self.config.data_path)

        optimizer = get_optimizer_with_custom_lr_sheduler(embedding_dim=self.config.params_embedding_dim)
        self.model.compile(optimizer=optimizer, loss=MaskedLoss(), metrics=[masked_acc])
        train_steps = data['train'].num_rows//self.config.params_batch_size
        
        X_train = np.concatenate((data['train']['input_ids'],data['train']['attention_mask']),axis=-1)
        y_train = np.array(data['train']['labels'])

        X_val = np.concatenate((data['validation']['input_ids'],data['validation']['attention_mask']),axis=-1)
        y_val = np.array(data['validation']['labels'])

        self.model.fit(
            X_train,y_train,
            train_steps=train_steps,
            validation_data=(X_val,y_val),
            epochs=self.config.params_epochs
        )

        self.save_model_weights(os.path.join(self.config.data_path,'model'),self.model)
        


    @staticmethod
    def save_model_weights(path: Path, model: tf.keras.models.Model):
        model.save_weights(path)

    def load_tokenizer(self):
        with open(self.config.tokenizer_path, 'rb') as f:
            tokenizer = pickle.load(f)
        return tokenizer

In [45]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.get_model()
    model_trainer.train()

except Exception as e:
    raise e

[2024-04-24 00:05:17,647: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-24 00:05:17,653: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-24 00:05:17,654: INFO: common: created directory at: artifacts]
[2024-04-24 00:05:17,655: INFO: common: created directory at: artifacts/model_trainer]
DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


KeyError: "Column input_ids not in the dataset. Current columns in the dataset: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags']"